In [1]:
import pandas as pd

data = [
    {
        "name": "Boho Breeze Maxi Dress",
        "desc": "Flowy maxi dress in earthy tones with tassels and paisley patterns for a free-spirited festival vibe.",
        "vibes": ["boho", "festival", "earthy"]
    },
    {
        "name": "Urban Edge Denim Jacket",
        "desc": "Cropped black denim jacket with metallic hardware for an energetic, urban streetwear look.",
        "vibes": ["urban", "streetwear", "edgy"]
    },
    {
        "name": "Cozy Cloud Knit Set",
        "desc": "Ultra-soft knit co-ord in beige for Netflix-and-chill comfort and minimal cozy vibes.",
        "vibes": ["cozy", "minimal", "soft"]
    },
    {
        "name": "Athflow Runner Sneakers",
        "desc": "Lightweight sneakers designed for athleisure comfort with breathable mesh and cushioning.",
        "vibes": ["sporty", "functional", "athleisure"]
    },
    {
        "name": "Desi Glow Kurta Set",
        "desc": "Cotton kurta set with subtle zari work for a festive Indian ethnic glow.",
        "vibes": ["festive", "ethnic", "traditional"]
    },
]

df = pd.DataFrame(data)
df


,name,desc,vibes
0,Boho Breeze Maxi Dress,Flowy maxi dress in earthy tones with tassels ...,"[boho, festival, earthy]"
1,Urban Edge Denim Jacket,Cropped black denim jacket with metallic hardw...,"[urban, streetwear, edgy]"
2,Cozy Cloud Knit Set,Ultra-soft knit co-ord in beige for Netflix-an...,"[cozy, minimal, soft]"
3,Athflow Runner Sneakers,Lightweight sneakers designed for athleisure c...,"[sporty, functional, athleisure]"
4,Desi Glow Kurta Set,Cotton kurta set with subtle zari work for a f...,"[festive, ethnic, traditional]"
